# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for DatAMRame functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DatAMRame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## AMRICAN

## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [13]:
# Check directory where AMP-PD data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AMR/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/chr9_AMR_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/chr9_AMR_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/chr9_AMR_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/chr9_AMR_release3.pvar...
- [4/4 files][163.2 MiB/163.2 MiB] 100% Done                                    
Operation completed over 4 objects/163.2 MiB.                                    


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [22]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [23]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [24]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_AMR_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_AMR

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AMR.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_AMR
  --pfile chr9_AMR_release3
  --to-bp 129148946

Start time: Fri Jun  2 07:20:31 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
417 samples (186 females, 231 males; 417 founders) loaded from
chr9_AMR_release3.psam.
448914 variants loaded from chr9_AMR_release3.pvar.
1 binary phenotype loaded (230 cases, 182 controls).
178 variants remaining after main filters.
Writing pheno_PP2A_AMR.fam ... done.
Writing pheno_PP2A_AMR.bim ... done.
Writing pheno_PP2A_AMR.bed ... 0%done.
End time: Fri Jun  2 07:20:31 2023


### Visualize plink files bim, fam and bed

In [25]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AMR.bim

9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111059:G:A	0	129111059	A	G
9	chr9:129111157:A:T	0	129111157	T	A
9	chr9:129111184:G:C	0	129111184	C	G
9	chr9:129111392:G:C	0	129111392	C	G
9	chr9:129111962:C:T	0	129111962	T	C
9	chr9:129112048:G:A	0	129112048	A	G
9	chr9:129112273:A:T	0	129112273	T	A
9	chr9:129112362:A:G	0	129112362	G	A
9	chr9:129112540:C:T	0	129112540	T	C


In [26]:
%%bash 
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

grep -e "CORIELL_000189_s1" -e "UMD_000275_s1" master_key_release3_final.csv > GP2_AMR_comphet.txt
head GP2_AMR_comphet.txt

CORIELL_000189,CORIELL_000189_s1,m3,PD,2,PD,2,63.0,42.0,,,Unknown,No,USA,CORIELL,0,,AMR,
UMD_000275,UMD_000275_s1,m1,PD,2,PD,1,,,,,Not Reported,No,USA,UMD,0,,AMR,


In [27]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AMR.fam

0	APGS_000157_s1	0	0	1	2
0	APGS_000551_s1	0	0	1	2
0	APGS_000609_s1	0	0	1	2
0	APGS_000724_s1	0	0	1	2
0	APGS_000981_s1	0	0	1	2
0	APGS_001003_s1	0	0	2	2
0	BCM_000007_s1	0	0	1	2
0	BCM_000020_s1	0	0	1	2
0	BCM_000054_s1	0	0	2	2
0	BCM_000127_s1	0	0	1	2


In [28]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AMR.bed

l묺������>���������/Ϗ����������뺻���>���˿뿫�������������������+���O���������*���ﯯ.�� ��������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������                       �                                                       ( �          �������������������������������������������������������������������������������������������������������������*
������8�"�0�2���`�#�
��8(��
�"(���:�,���*���8�*
����
�0*�((�#숸��(����/����ꈢ�"���(�:�����*������(�"�0�2���`�#�
��8(��� (���:�,���*���8�*
����
�0*�((�#숸��(��������ꈢ�"���(�8���������������������������������������������������������������������������������

In [29]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_AMR \
--recode vcf-fid \
--out pheno_PP2A_AMR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AMR.log.
Options in effect:
  --bfile pheno_PP2A_AMR
  --out pheno_PP2A_AMR
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%

## Annotate PP2A variants

In [30]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A_AMR.vcf

tabix -f -p vcf pheno_PP2A_AMR.vcf.gz
tabix -f -p vcf pheno_PP2A_AMR.vcf.gz

[bgzip] can't create pheno_PP2A_AMR.vcf.gz: File exists


In [31]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AMR.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A_AMR.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A_AMR.vcf.gz > pheno_PP2A_AMR.annovar.avinput>
NOTICE: Finished reading 185 lines from VCF file
NOTICE: A total of 178 locus in VCF file passed QC threshold, representing 163 SNPs (121 transitions and 42 transversions) and 15 indels/substitutions
NOTICE: Finished writing allele frequencies based on 67971 SNP genotypes (50457 transitions and 17514 transversions) and 6255 indels/substitutions for 417 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AMR.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A_AMR.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -

## Extract coding and non-syn variants

In [32]:
# Visualize multianno file
PP2A_AMR = pd.read_csv(f'{WORK_DIR}/pheno_PP2A_AMR.annovar.hg38_multianno.txt', sep = '\t')
PP2A_AMR.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo420,Otherinfo421,Otherinfo422,Otherinfo423,Otherinfo424,Otherinfo425,Otherinfo426,Otherinfo427,Otherinfo428,Otherinfo429
0,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/0,0/1,0/1,0/0,0/1,0/0,0/1,1/1,0/0,1/1
1,9,129111059,129111059,G,A,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129111157,129111157,A,T,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,9,129111184,129111184,G,C,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,9,129111392,129111392,G,C,UTR5,PTPA,NM_178001:c.-209G>C;NM_001271832:c.-209G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [33]:
PP2A_AMR.count()

Chr             178
Start           178
End             178
Ref             178
Alt             178
               ... 
Otherinfo425    178
Otherinfo426    178
Otherinfo427    178
Otherinfo428    178
Otherinfo429    178
Length: 440, dtype: int64

In [34]:
# Filter exonic variants
coding_AMR = PP2A_AMR[PP2A_AMR['Func.refGene'] == 'exonic']
coding_AMR.count()
coding_AMR.head(5)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo420,Otherinfo421,Otherinfo422,Otherinfo423,Otherinfo424,Otherinfo425,Otherinfo426,Otherinfo427,Otherinfo428,Otherinfo429
108,9,129134852,129134852,G,A,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0
170,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0,0/0


In [35]:
# Filter exonic and non-syn vars
coding_nonsynonymous_AMR = PP2A_AMR[(PP2A_AMR['Func.refGene'] == 'exonic') & (PP2A_AMR['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_AMR.count(1)

108    440
170    440
dtype: int64

In [36]:
coding_nonsynonymous_AMR.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo420,Otherinfo421,Otherinfo422,Otherinfo423,Otherinfo424,Otherinfo425,Otherinfo426,Otherinfo427,Otherinfo428,Otherinfo429
108,9,129134852,129134852,G,A,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0


In [37]:
print(coding_nonsynonymous_AMR)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
108    9  129134852  129134852   G   A       exonic         PTPA   
170    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
108                  .  nonsynonymous SNV   
170                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo420  \
108  PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...  ...          0/0   
170  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...          0/0   

     Otherinfo421 Otherinfo422 Otherinfo423  Otherinfo424  Otherinfo425  \
108           0/0          0/0          0/0           0/0           0/0   
170           0/0          0/1          0/0           0/0           0/0   

    Otherinfo426 Otherinfo427 Otherinfo428 Otherinfo429  
108          0/0          0/1          0/0          0/0  
170          0/0          0/1          0/0          0/0  

[2 rows x 440 columns]


In [38]:
coding_nonsynonymous_AMR.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_AMR.head(9)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo420,Otherinfo421,Otherinfo422,Otherinfo423,Otherinfo424,Otherinfo425,Otherinfo426,Otherinfo427,Otherinfo428,Otherinfo429
108,9,129134852,129134852,G,A,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0
170,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0,0/0


In [39]:
# Filter intronic variants
intronic_AMR = PP2A_AMR[PP2A_AMR['Func.refGene'] == 'intronic']
intronic_AMR.count()

Chr             167
Start           167
End             167
Ref             167
Alt             167
               ... 
Otherinfo425    167
Otherinfo426    167
Otherinfo427    167
Otherinfo428    167
Otherinfo429    167
Length: 440, dtype: int64

In [40]:
# Filter UTR3 variants
UTR3_AMR = PP2A_AMR[PP2A_AMR['Func.refGene'] == ('UTR3')]
UTR3_AMR.count()

Chr             7
Start           7
End             7
Ref             7
Alt             7
               ..
Otherinfo425    7
Otherinfo426    7
Otherinfo427    7
Otherinfo428    7
Otherinfo429    7
Length: 440, dtype: int64

In [41]:
# Filter UTR5 variants
UTR5_AMR = PP2A_AMR[PP2A_AMR['Func.refGene'] == ('UTR5')]
UTR5_AMR.count()

Chr             2
Start           2
End             2
Ref             2
Alt             2
               ..
Otherinfo425    2
Otherinfo426    2
Otherinfo427    2
Otherinfo428    2
Otherinfo429    2
Length: 440, dtype: int64

In [42]:
# Calculate freq - cases vs controls

In [43]:
reduced_coding_nonsynonymous_AMR = coding_nonsynonymous_AMR[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_AMR.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_AMR.txt', sep = '\t', index = False, header= False)

In [44]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_AMR.txt

9	129134852	129134852	PTPA
9	129147457	129147457	PTPA


In [45]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AMR --extract range reduced_coding_nonsynonymous_AMR.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AMR --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AMR.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AMR
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_AMR.txt
  --out coding_nonsynonymous_pheno_PP2A_AMR

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
--extract range: 176 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%3

In [46]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AMR --extract range reduced_coding_nonsynonymous_AMR.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_AMR_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_AMR_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_AMR
  --extract range reduced_coding_nonsynonymous_AMR.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_AMR_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
--extract range: 176 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%3

In [47]:
PP2A_AMR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AMR.assoc', delim_whitespace=True)
PP2A_AMR_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129134852:G:A,129134852,A,0.01739,0.005495,G,2.3990,0.1214,3.2040,0.7937,0.6761,15.180
1,9,chr9:129147457:C:T,129147457,T,0.04565,0.058010,C,0.6382,0.4244,0.7768,0.3169,0.4174,1.446


In [48]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AMR --extract range reduced_coding_nonsynonymous_AMR.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AMR --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AMR.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AMR
  --extract range reduced_coding_nonsynonymous_AMR.txt
  --out coding_nonsynonymous_pheno_PP2A_AMR

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
--extract range: 176 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33

In [49]:
PP2A_AMR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AMR.assoc.adjusted', delim_whitespace=True)
PP2A_AMR_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129134852:G:A,0.1214,0.3960,0.2428,0.2428,0.2281,0.2281,0.2428,0.3642
1,9,chr9:129147457:C:T,0.4244,0.6616,0.8487,0.4244,0.6686,0.4244,0.4244,0.6365


## Calculate freq of HMZ in cases versus controls

In [50]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AMR --extract range reduced_coding_nonsynonymous_AMR.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AMR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AMR.log.
Options in effect:
  --bfile pheno_PP2A_AMR
  --extract range reduced_coding_nonsynonymous_AMR.txt
  --out coding_nonsynonymous_pheno_PP2A_AMR
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
--extract range: 176 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [51]:
PP2A_AMR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AMR.assoc', delim_whitespace=True)
PP2A_AMR_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
0,9,chr9:129134852:G:A,129134852,A,0.01739,0.005495,G,2.3990,0.1214,3.2040
1,9,chr9:129147457:C:T,129147457,T,0.04565,0.058010,C,0.6382,0.4244,0.7768


In [52]:
PP2A_AMR_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AMR.raw', delim_whitespace=True)
PP2A_AMR_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
0,0,APGS_000157_s1,0,0,1,2,0,0.0
1,0,APGS_000551_s1,0,0,1,2,0,0.0
2,0,APGS_000609_s1,0,0,1,2,0,0.0
3,0,APGS_000724_s1,0,0,1,2,0,0.0
4,0,APGS_000981_s1,0,0,1,2,0,1.0


In [53]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_PP2A_AMR --extract range reduced_coding_nonsynonymous_AMR.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AMR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AMR.log.
Options in effect:
  --bfile pheno_PP2A_AMR
  --extract range reduced_coding_nonsynonymous_AMR.txt
  --out coding_nonsynonymous_pheno_PP2A_AMR
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
178 variants loaded from .bim file.
417 people (231 males, 186 females) loaded from .fam.
412 phenotype values loaded from .fam.
--extract range: 176 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 417 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [54]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AMR_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_AMR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T


In [55]:
PP2A_hom_casos_AMR.shape

(0, 8)

In [56]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AMR_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_AMR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T


In [57]:
PP2A_hom_controles_AMR.shape

(0, 8)

In [58]:
# Explore HMZs for chr9:129134852:G:A_A in cases
PP2A_hom_casos_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129134852:G:A_A'] == 2) & (PP2A_AMR_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_AMR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T


In [59]:
# Explore HMZs for chr9:129134852:G:A_A in controls
PP2A_hom_controles_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129134852:G:A_A'] == 2) & (PP2A_AMR_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_AMR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T


In [60]:
# Identifying individuals het for the alternative allele
het = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129147457:C:T_T'] == 1) | (PP2A_AMR_recode['chr9:129134852:G:A_A'] == 1)]
het = het.dropna()
print(het.shape)

(50, 8)


In [61]:
het.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
4,0,APGS_000981_s1,0,0,1,2,0,1.0
8,0,BCM_000054_s1,0,0,2,2,0,1.0
9,0,BCM_000127_s1,0,0,1,2,0,1.0
11,0,BCM_000136_s1,0,0,1,2,0,1.0
15,0,BCM_000180_s1,0,0,2,1,0,1.0


In [62]:
het['compound'] = het['chr9:129147457:C:T_T'] + het['chr9:129134852:G:A_A']
het['compound'].value_counts()

1.0    47
2.0     3
Name: compound, dtype: int64

In [63]:
comp_het_info_cases = het[(het['PHENOTYPE'] == 2) & (het['compound'] == 2)]
print(comp_het_info_cases.shape)

(2, 9)


In [64]:
comp_het_info_control = het[(het['PHENOTYPE'] == 1) & (het['compound'] == 2)]
print(comp_het_info_control.shape)

(1, 9)


In [65]:
comp_het_info_cases.to_csv('/home/jupyter/PTPA/comp_het_info_cases.txt', sep = '\t', index=True)
comp_het_info_cases.head(54)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T,compound
90,0,CORIELL_000189_s1,0,0,2,2,1,1.0,2.0
414,0,UMD_000275_s1,0,0,1,2,1,1.0,2.0


In [66]:
comp_het_info_control.to_csv('/home/jupyter/PTPA/comp_het_info_control.txt', sep = '\t', index=True)
comp_het_info_control.head(54)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T,compound
270,0,CORIELL_006462_s1,0,0,1,1,1,1.0,2.0


In [67]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AMR_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_AMR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
4,0,APGS_000981_s1,0,0,1,2,0,1.0
8,0,BCM_000054_s1,0,0,2,2,0,1.0
9,0,BCM_000127_s1,0,0,1,2,0,1.0
11,0,BCM_000136_s1,0,0,1,2,0,1.0
19,0,BCM_000206_s1,0,0,1,2,0,1.0
20,0,BCM_000214_s1,0,0,1,2,0,1.0
90,0,CORIELL_000189_s1,0,0,2,2,1,1.0
130,0,CORIELL_001342_s1,0,0,1,2,0,1.0
143,0,CORIELL_001771_s1,0,0,2,2,0,1.0
147,0,CORIELL_002148_s1,0,0,2,2,0,1.0


In [68]:
# Explore HETs for chr9:129147457:C:T_T in controls
PP2A_het_controles_129147457_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AMR_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_AMR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
15,0,BCM_000180_s1,0,0,2,1,0,1.0
66,0,BCM_000641_s1,0,0,2,1,0,1.0
255,0,CORIELL_005578_s1,0,0,1,1,0,1.0
270,0,CORIELL_006462_s1,0,0,1,1,1,1.0
271,0,CORIELL_006464_s1,0,0,1,1,0,1.0
283,0,CORIELL_006511_s1,0,0,1,1,0,1.0
297,0,CORIELL_006609_s1,0,0,1,1,0,1.0
299,0,CORIELL_006634_s1,0,0,2,1,0,1.0
306,0,CORIELL_006647_s1,0,0,1,1,0,1.0
313,0,CORIELL_006662_s1,0,0,2,1,0,1.0


In [95]:
PP2A_het_casos_129147457_AMR.to_csv('PP2A_het_casos_129147457_AMR.csv')

In [96]:
temp = pd.read_csv('PP2A_het_casos_129147457_AMR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
0,4,0,APGS_000981_s1,0,0,1,2,0,1.0
1,8,0,BCM_000054_s1,0,0,2,2,0,1.0
2,9,0,BCM_000127_s1,0,0,1,2,0,1.0
3,11,0,BCM_000136_s1,0,0,1,2,0,1.0
4,19,0,BCM_000206_s1,0,0,1,2,0,1.0
5,20,0,BCM_000214_s1,0,0,1,2,0,1.0
6,90,0,CORIELL_000189_s1,0,0,2,2,1,1.0
7,130,0,CORIELL_001342_s1,0,0,1,2,0,1.0
8,143,0,CORIELL_001771_s1,0,0,2,2,0,1.0
9,147,0,CORIELL_002148_s1,0,0,2,2,0,1.0


In [97]:
PP2A_het_controles_129147457_AMR.to_csv('PP2A_het_controles_129147457_AMR.csv')

In [98]:
temp = pd.read_csv('PP2A_het_controles_129147457_AMR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
0,15,0,BCM_000180_s1,0,0,2,1,0,1.0
1,66,0,BCM_000641_s1,0,0,2,1,0,1.0
2,255,0,CORIELL_005578_s1,0,0,1,1,0,1.0
3,270,0,CORIELL_006462_s1,0,0,1,1,1,1.0
4,271,0,CORIELL_006464_s1,0,0,1,1,0,1.0
5,283,0,CORIELL_006511_s1,0,0,1,1,0,1.0
6,297,0,CORIELL_006609_s1,0,0,1,1,0,1.0
7,299,0,CORIELL_006634_s1,0,0,2,1,0,1.0
8,306,0,CORIELL_006647_s1,0,0,1,1,0,1.0
9,313,0,CORIELL_006662_s1,0,0,2,1,0,1.0


In [73]:
# Explore HETs for chr9:129134852:G:A_A in cases
PP2A_het_casos_129134852_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129134852:G:A_A'] == 1) & (PP2A_AMR_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129134852_AMR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
90,0,CORIELL_000189_s1,0,0,2,2,1,1.0
107,0,CORIELL_000589_s1,0,0,2,2,1,0.0
124,0,CORIELL_001146_s1,0,0,1,2,1,0.0
163,0,CORIELL_003334_s1,0,0,2,2,1,0.0
195,0,CORIELL_004207_s1,0,0,2,2,1,0.0
198,0,CORIELL_004245_s1,0,0,2,2,1,0.0
205,0,CORIELL_004338_s1,0,0,2,2,1,0.0
414,0,UMD_000275_s1,0,0,1,2,1,1.0


In [74]:
# Explore HETs for chr9:129134852:G:A_A in controles
PP2A_het_controles_129134852_AMR = PP2A_AMR_recode[(PP2A_AMR_recode['chr9:129134852:G:A_A'] == 1) & (PP2A_AMR_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129134852_AMR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
47,0,BCM_000529_s1,0,0,2,1,1,0.0
270,0,CORIELL_006462_s1,0,0,1,1,1,1.0


In [87]:
PP2A_het_casos_129134852_AMR.to_csv('PP2A_het_casos_129134852_AMR.csv')

In [88]:
temp = pd.read_csv('PP2A_het_casos_129134852_AMR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
0,90,0,CORIELL_000189_s1,0,0,2,2,1,1.0
1,107,0,CORIELL_000589_s1,0,0,2,2,1,0.0
2,124,0,CORIELL_001146_s1,0,0,1,2,1,0.0
3,163,0,CORIELL_003334_s1,0,0,2,2,1,0.0
4,195,0,CORIELL_004207_s1,0,0,2,2,1,0.0
5,198,0,CORIELL_004245_s1,0,0,2,2,1,0.0
6,205,0,CORIELL_004338_s1,0,0,2,2,1,0.0
7,414,0,UMD_000275_s1,0,0,1,2,1,1.0


In [92]:
PP2A_het_controles_129134852_AMR.to_csv('PP2A_het_controles_129134852_AMR.csv')

In [93]:
temp = pd.read_csv('PP2A_het_controles_129134852_AMR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134852:G:A_A,chr9:129147457:C:T_T
0,47,0,BCM_000529_s1,0,0,2,1,1,0.0
1,270,0,CORIELL_006462_s1,0,0,1,1,1,1.0


In [79]:
!pip install rpy2
%load_ext rpy2.ipython

In [80]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [81]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

data.table 1.14.6 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [82]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [99]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_AMR <- fread("PP2A_het_casos_129147457_AMR.csv", header =T)
PP2A_het_casos_129147457_AMR
temp <- merge(covs, PP2A_het_casos_129147457_AMR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_AMR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_AMR_extracted <- fread("PP2A_het_casos_129147457_AMR_extracted.txt", header =T)
PP2A_het_casos_129147457_AMR_extracted

                  IID          GP2ID manifest_id phenotype pheno_for_qc
 1:    APGS_000981_s1    APGS_000981          m1        PD            2
 2:     BCM_000054_s1     BCM_000054          m1        PD            2
 3:     BCM_000127_s1     BCM_000127          m1        PD            2
 4:     BCM_000136_s1     BCM_000136          m1        PD            2
 5:     BCM_000206_s1     BCM_000206          m1        PD            2
 6:     BCM_000214_s1     BCM_000214          m1        PD            2
 7: CORIELL_000189_s1 CORIELL_000189          m3        PD            2
 8: CORIELL_001342_s1 CORIELL_001342          m3        PD            2
 9: CORIELL_001771_s1 CORIELL_001771          m3        PD            2
10: CORIELL_002148_s1 CORIELL_002148          m3        PD            2
11: CORIELL_003627_s1 CORIELL_003627          m3        PD            2
12: CORIELL_003890_s1 CORIELL_003890          m3        PD            2
13: CORIELL_004101_s1 CORIELL_004101          m3        PD      

In [100]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_AMR <- fread("PP2A_het_controles_129147457_AMR.csv", header =T)
PP2A_het_controles_129147457_AMR
temp <- merge(covs, PP2A_het_controles_129147457_AMR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129147457_AMR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_AMR_extracted <- fread("PP2A_het_controles_129147457_AMR_extracted.txt", header =T)
PP2A_het_controles_129147457_AMR_extracted

                  IID          GP2ID manifest_id phenotype pheno_for_qc
 1:     BCM_000180_s1     BCM_000180          m1   Control            1
 2:     BCM_000641_s1     BCM_000641          m1   Control            1
 3: CORIELL_005578_s1 CORIELL_005578          m6   Control            1
 4: CORIELL_006462_s1 CORIELL_006462          m6   Control            1
 5: CORIELL_006464_s1 CORIELL_006464          m6   Control            1
 6: CORIELL_006511_s1 CORIELL_006511          m6   Control            1
 7: CORIELL_006609_s1 CORIELL_006609          m6   Control            1
 8: CORIELL_006634_s1 CORIELL_006634          m6   Control            1
 9: CORIELL_006647_s1 CORIELL_006647          m6   Control            1
10: CORIELL_006662_s1 CORIELL_006662          m6   Control            1
11: CORIELL_006663_s1 CORIELL_006663          m6   Control            1
12: CORIELL_006692_s1 CORIELL_006692          m6   Control            1
13: CORIELL_006701_s1 CORIELL_006701          m6   Control      

In [89]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129134852_AMR <- fread("PP2A_het_casos_129134852_AMR.csv", header =T)
PP2A_het_casos_129134852_AMR
temp <- merge(covs, PP2A_het_casos_129134852_AMR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129134852_AMR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129134852_AMR_extracted <- fread("PP2A_het_casos_129134852_AMR_extracted.txt", header =T)
PP2A_het_casos_129134852_AMR_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1: CORIELL_000189_s1 CORIELL_000189          m3        PD            2
2: CORIELL_000589_s1 CORIELL_000589          m3        PD            2
3: CORIELL_001146_s1 CORIELL_001146          m3        PD            2
4: CORIELL_003334_s1 CORIELL_003334          m3        PD            2
5: CORIELL_004207_s1 CORIELL_004207          m3        PD            2
6: CORIELL_004245_s1 CORIELL_004245          m3        PD            2
7: CORIELL_004338_s1 CORIELL_004338          m4        PD            2
8:     UMD_000275_s1     UMD_000275          m1        PD            2
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:          PD          2  63           42               NA           NA
2:          PD          2  42           38               NA           NA
3:          PD          1  63           60               NA           NA
4:          PD          2  60           52               52          

In [94]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129134852_AMR <- fread("PP2A_het_controles_129134852_AMR.csv", header =T)
PP2A_het_controles_129134852_AMR
temp <- merge(covs, PP2A_het_controles_129134852_AMR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129134852_AMR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129134852_AMR_extracted <- fread("PP2A_het_controles_129134852_AMR_extracted.txt", header =T)
PP2A_het_controles_129134852_AMR_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1:     BCM_000529_s1     BCM_000529          m1   Control            1
2: CORIELL_006462_s1 CORIELL_006462          m6   Control            1
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:     Control          2  NA           NA               NA           NA
2:     Control          1  59           NA               NA           NA
   race_for_qc family_history_for_qc region_for_qc   study pruned pruned_reason
1:     Unknown                    No           USA     BCM      0            NA
2:       Other                    No           USA CORIELL      0            NA
   label related  V1 FID PAT MAT SEX PHENOTYPE chr9:129134852:G:A_A
1:   AMR      NA  47   0   0   0   2         1                    1
2:   AMR      NA 270   0   0   0   1         1                    1
   chr9:129147457:C:T_T
1:                    0
2:                    1


## Save out results..!

In [101]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PP2A_GP2/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_controles_129147457_EUR_extracted.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AJ_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_CAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AFR.fam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AMR.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous_AFR.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.annovar.hg38_multianno.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous_AMR.txt [Content-Typ